# LyricistAI

## Sequence, Sequential?

* Sequence data : 나열된 데이터(비슷한 특징을 가지거나 같은 속성이여야 하는 것은 아니다.)
* 다만 인공지능에서는 예측을 위해서 data들이 서로 유사한 특징을 가져야한다. 이러한 데이터를 Sequential Data라 한다.
* AI가 문법을 익히기는 힘드니 통계적인 방법을 사용한다.

## 문장 만들기

### 문장 연결

* 수많은 글을 읽게 함으로써 다음 단어를 예측을 잘 할 수 있게 한다.
* start 를 주고 나온 결과를 다음 뉴런의 input으로 사용을 반복하고 end가 나옴 이러한 순환적 특징 때문에 순환 신경망(RNN)이라한다.

In [1]:
sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


### 언어 모델

* 단어 다음에 무슨 단어가 나올지 예측하는 확률 모델을 언어 모델이라 한다.
* P(w<sub>n</sub>​∣w<sub>1</sub> ,...,w<sub>n−1</sub>; θ)

## 데이터 다듬기

### 데이터 불러오기

* 파일을 읽기로 열고 라인 단위로 끊어서 list 형태로 읽어옵니다.
* 앞에서부터 10라인만 화면에 출력해 볼까요?

In [2]:
import os, re 
import numpy as np
import tensorflow as tf

# file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
file_path = 'data/shakespeare.txt'

with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


### 데이터 전처리 : 공백과 화자 제거

* 여기서 공백인 문장과 화자가 표시된 문장을 삭제해주고 싶습니다.
* 길이가 0인 문장과 문장의 끝이 : 인 문장은 건너뜁니다.
* 그리고 문장을 10개만 확인합니다.

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0:
        continue
    if sentence[-1] == ":":
        continue
    if idx > 9:
        break
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


토근화(Tokenize) : 문장을 일정한 기준으로 쪼개서 단어로 만드는 것

### 데이터 전처리 함수 설정 - 단어

1. 소문자로 바꾸고(lower) 양쪽 공백을 지웁니다(strip).
2. 특수 문자 양쪽에 공백을 넣습니다.
3. 여러 개의 공백을 하나로 만듭니다.
4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다.
5. 다시 양쪽 공백을 지웁니다.
6. 문장의 시작에는 <start>, 끝에는 <end>를 추가합니다.

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


### 데이터 전처리 함수 적용

* 문장의 길이가 0이거나 문자의 마지막이 : 인 경우는 제외합니다.
* 문장에 전처리 함수를 적용하고 저장합니다.

In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if sentence[-1] == ":":
        continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

### Tokenizer 함수 설정

* 자연어 처리용 모듈 : tf.keras.preprocessing.text.Tokenizer
* 벡터화(vectorize) : 정제된 데이터를 토큰화 -> 단어 사전 생성 -> 데이터를 숫자로 변환
* tensor : 숫자로 변환된 데이터

링크 참조  
* [Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) : 토큰화 관련 클래스
* [pad_sequences](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) : 시퀀스를 같은 길이가 되도록 pad 합니다.

Tokenizer args
* num_words : 저장할 단어의 개수
* filters : 필터링 될 문자열(제거할 구두점 같은 것들), default 는 '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n' 이다.
* lower : 텍스트를 소문자로 만들 것인가? - boolean
* split : 단어를 나눌 기준(스페이스바 단위로 나눈다고 할때 ' ' 넣기)
* char_level : True 면 모든 캐릭터가 token화가 된다.
* oov_token : 적용하면 단어 index가 생기고 단어 사전에 저장되지 않는 값들을 이것으로 바꾼다(text_to_sequence call 동안)
* analyzer : 함수, 텍스트를 나눌 특정 분석. 기본 분석은 text_to_word_sequence 이다.  

Tokenizer method
* fit_on_sequences : sequence 에 따라 사전 최신화
* fit_on_texts : texts 에 따라 사전 최신화
* text_to_sequences : texts 를 정수 시퀀스로 변환

pad_sequences args
* sequence : 변환할 시퀀스
* maxlen : 최대 길이
* dtype : 데이터 타입(default : int32)
* padding : pre or past(pre: 앞에서부터 채움, past: 뒤에서부터 채움, pre 패딩이 더 좋다는 중론, default = pre)
* truncating : pre or past(maxlen보다 클 경우 앞부분을 지울지 뒷부분을 지울지 결정, default = pre)
* value : padding시 넣는 값(default : 0)

In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras.preprocessing.text.Tokenizer object at 0x7f442c5a64a0>


In [7]:
tensor[:3]

array([[   2,  143,   40,  933,  140,  591,    4,  124,   24,  110,    5,
           3,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   2,  110,    4,  110,    5,    3,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   2,   11,   50,   43, 1201,  316,    9,  201,   74,    9, 3034,
          15,    3,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [8]:
for idx in tokenizer.index_word.values():
    if idx == 0:
        print('Find 0!')

In [9]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


* 단어사전의 key = 1 에 사전에 들어가지 못한 단어들이 들어가며 value 는 <'unk'> 입니다.
* 사이즈에 맞춰서 뒤에 0이 생기지만 결국 마지막은 3(<'end'>)이며 시작은 2(<'start'>)입니다.
* 그렇기에 마지막 값은 <'pad'>일 가능성이 높습니다.

### 소스 및 타겟 문장 생성

In [10]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

print(raw_corpus[:9])
print(src_input[0])
print(tgt_input[0])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']
[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


소스 문장은 맨뒤를 짜름.  
모델 코드에서 계산 할 때 시간 단축을 위해 뒤에 있는 0을 건너 뛰도록 end 나오면 종료함.  
그러므로 소스 문장은 end 부분을 짜르는게 아닌 마지막 0 부분을 짜름.  
만약 가장 긴 문장이여서 잘리는 부분이 0이 아닌 end라도 <'start'>, <'end'> 둘 다 내가 추가해준 것이므로 그냥 삭제.  

타겟 문장은 정답인 문장이므로 맨앞을 짜름.  
지금은 padding을 'post'로 했으니 그렇지만 'pre'로 하게되면 결국 0이 앞에 생김,  
이 경우에도 <'start'> 가 아닌 padding 값을 자르게 될 것.  
결국 그냥 한칸씩만 비워주는 개념으로 생각.

### Dataset 사용

[tensorflow Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

기존의 numpy array 로 train/test 나눠서 훈련 시키는게 아닌  
텐서로 만들어진 data를 이용해 Dataset 객체를 생성할 것.  
이미 데이터가 tensor 형태이므로 slice를 통해 객체 생성

In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

print(type(src_input), type(tgt_input))

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(type(dataset))
dataset

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


2022-08-08 10:01:22.843378: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 10:01:22.914787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 10:01:22.914952: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-08 10:01:22.917021: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

<BatchDataset element_spec=(TensorSpec(shape=(256, 20), dtype=tf.int32, name=None), TensorSpec(shape=(256, 20), dtype=tf.int32, name=None))>

* 소스 문장의 크기만큼 버퍼 사이즈를 잡는다.
* buffer : 전송하는 동안 그 데이터를 보관하는 메모리 영역
* batch size : 일괄처리량
* step
* 단어장의 전체 크기 : 원래 크기 + <'pad'>인 0 = 원래 크기 + 1
* 주어진 tensor 들로 나눠진 Dataset 만들기
* 해당 사이즈 만큼 셔플
* BATCH_SIZE 잡고, drop_remainder : 마지막에 batchsize보다 작은게 나오면 batch를 drop 함.

* 정규표현식을 이용한 corpus 생성
* tf.keras.preprocessing.text.Tokenizer를 이용해 corpus를 텐서로 변환
* tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서를 tf.data.Dataset객체로 변환  
  를 통해 데이터 다듬기 과정이 끝났습니다.

## 인공지능 학습

모델을 1개의 Embedding, 2개의 LSTM, 1개의 Dense Layer 들로 구성 할 것.

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

* embedding_size : 단어 벡터의 차원 수
* hidden_size : hidden layer 깊이 같은데?

dataset 에서 배치 1개만 불러와서 모델에 넣기

In [13]:
for src_sample, tgt_sample in dataset.take(1):
    break

model(src_sample)

2022-08-08 10:01:25.849791: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204


<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 5.27822005e-04,  4.00538935e-04, -3.24389694e-04, ...,
          2.38754656e-04,  1.72172106e-04, -1.27445412e-04],
        [ 1.02338195e-03,  6.99171738e-04, -2.87780509e-04, ...,
          3.58307734e-04,  4.27360705e-04, -3.67908884e-04],
        [ 1.15139468e-03,  7.27311999e-04, -5.54709055e-04, ...,
          7.97548331e-04,  9.88477026e-04, -1.38388554e-04],
        ...,
        [ 8.31462035e-04, -9.24486085e-04,  4.09787812e-04, ...,
          2.46728491e-03, -4.07283660e-03, -7.44014746e-04],
        [ 8.83519766e-04, -1.07495172e-03,  7.32855930e-04, ...,
          2.82456540e-03, -4.53445874e-03, -5.97644306e-04],
        [ 9.49400885e-04, -1.19806943e-03,  1.05003337e-03, ...,
          3.17598321e-03, -4.95734671e-03, -4.37945098e-04]],

       [[ 5.27822005e-04,  4.00538935e-04, -3.24389694e-04, ...,
          2.38754656e-04,  1.72172106e-04, -1.27445412e-04],
        [ 7.81894254e-04,  5.82162000e-04, -4.

* shape=(256, 20, 7001)
  - 7001은 단어의 개수이자 dense layer의 출력 수
  - 256은 이전에 지정해준 batch size, 배치를 하나만 가져왔기 때문
  - 20은 tf.keras.layers.LSTM(hidden_size, return_sequences=True)에서  
  - return_sequences=True를 해줬기 때문에 자신에게 입력된 시퀀스의 길이만큼 출력, False면 1개의 벡터만 출력

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1792256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  7176025   
                                                                 
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


이제 모델을 학습시켜봅시다.

In [15]:
# optimizer와 loss등은 차차 배웁니다 
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요

# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)

# model.fit() 함수에 들어가는 다양한 인자를 알고 싶다면 아래의 문서를 참고하세요. 
# https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit

model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 13s 118ms/step - loss: 3.5896
Epoch 2/30
93/93 [==============================] - 11s 119ms/step - loss: 2.8304
Epoch 3/30
93/93 [==============================] - 11s 120ms/step - loss: 2.7629
Epoch 4/30
93/93 [==============================] - 11s 121ms/step - loss: 2.6832
Epoch 5/30
93/93 [==============================] - 11s 121ms/step - loss: 2.6092
Epoch 6/30
93/93 [==============================] - 11s 121ms/step - loss: 2.5530
Epoch 7/30
93/93 [==============================] - 11s 122ms/step - loss: 2.4980
Epoch 8/30
93/93 [==============================] - 11s 122ms/step - loss: 2.4492
Epoch 9/30
93/93 [==============================] - 11s 122ms/step - loss: 2.4048
Epoch 10/30
93/93 [==============================] - 11s 122ms/step - loss: 2.3663
Epoch 11/30
93/93 [==============================] - 11s 122ms/step - loss: 2.3288
Epoch 12/30
93/93 [==============================] - 11s 122ms/step - loss: 2.2956
Epoch 13/30
9

## 잘 만들어졌는지 평가

* generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하게 합니다.

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

* generate_text() 함수에서 init_sentence를 인자로 받고는 있습니다. 이렇게 받은 인자를 일단 텐서로 만들고 있습니다. 디폴트로는 <start> 단어 하나만 받는군요.
* while의 첫 번째 루프에서 test_tensor에 <start> 하나만 들어갔다고 합시다. 우리의 모델이 출력으로 7001개의 단어 중 A를 골랐다고 합시다.
* while의 두 번째 루프에서 test_tensor에는 <start> A가 들어갑니다. 그래서 우리의 모델이 그다음 B를 골랐다고 합시다.
* while의 세 번째 루프에서 test_tensor에는 <start> A B가 들어갑니다. 그래서..... (이하 후략)

In [17]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s a <unk> , and i will not be so , <end> '